# DEMO Notebook
## Using imexam inside a Jupyter notebook with Ginga
### The Ginga HTML5 canvas viewer can be used as a viewer for the imexam package. While the Ginga viewer is active you have access to all the imexam as well as all the Ginga functions the HTML viewer provides. Some of the examples of Ginga functionality were taken directly from a Ginga demo notebook available on the Ginga github repo.

### This and other demo notebooks can be found on the SpaceTelescope github site: https://github.com/spacetelescope/imexam/

### If you are running with Python 3, ginga also requires that pillow be installed. You'll see a blank viewer popup in the html window if this is the case. You can "conda install pillow" and that should fix the problem.


In [ ]:
import imexam

### Start up a new tab with the ginga HTML5 viewer
#### You can disconnect the tab and place it anywhere else on your screen that you like

In [ ]:
a=imexam.connect(viewer='ginga')

In [ ]:
a.load_fits('iacs01t4q_flt.fits') #display fits image in a separate browser window

## When using a Ginga window the imexam() function is event driven. 
### This means that while looking at the image, pressing the "i" key will enter you into imexam mode. Subsequent pressing of the "q" key will put you back in regular viewing mode.

### While in imexam mode, the analysis keys are mapped to the imexam functions and the results will be returned in the notebook

In [ ]:
#No list of commands is printed with the event driven imexam, but you can always 
#see what the available commands are by issuing the imexam() call:
a.imexam()

<p align="left">The text printed above the plot: "read: e at x y" , means that the "e" key was pressed at those image coordinates. The "e" instructs imexam to make a contour plot.</p>



In [ ]:
# all of the plots are customizable; the parameters are stored in regular dictionaries
a.eimexam()

In [ ]:
#example of accessing the plot parameter dictionary directly for customization
a.set_plot_pars('e','title','My custom plot title')
a.set_plot_pars('e','floor',0.)
a.set_plot_pars('e','ncontours',4)

In [ ]:
a.imexam()

In [ ]:
# you can  save a copy of the current viewing window
a.window.ginga_view.show()

## Besides making plots you can also get basic aperture photometry using the "a" key and radial plots with "r"

In [ ]:
#You can also request a numpy array of the data being viewed:
data=a.get_data()

In [ ]:
data

In [ ]:
#you can look at the header as well
a.get_header()

## If you are using the ginga viewer, you can return the full Ginga image object and use any
of the methods which are enabled for it. You can look here for the Ginga quick reference: 
http://ginga.readthedocs.org/en/latest/quickref.html

In [ ]:
img=a.get_image()

In [ ]:
type(img)

In [ ]:
img.height, img.width, img.pixtoradec(100,100)

In [ ]:
img.pixtoradec(100,100)

In [ ]:
canvas=a.window.ginga_view.add_canvas()
canvas.delete_all_objects()
canvas.set_drawtype('rectangle')

## Now you can go to the viewer and draw a rectangle selection box, use the right mouse button

In [ ]:
a.window.ginga_view.show()

In [ ]:
from ginga.util import iqcalc
iq = iqcalc.IQCalc()

#find all the peaks in the rectangle area
r = canvas.objects[0]
data = img.cutout_shape(r)
peaks = iq.find_bright_peaks(data)

In [ ]:
peaks[:10] #show the last 10 peaks detected in the cutout

In [ ]:
objs = iq.evaluate_peaks(peaks, data)

In [ ]:
o1=objs[0]
o1

In [ ]:
# pixel coords are for cutout, so add back in origin of cutout
#  to get full data coords RA, DEC of first object
x1, y1, x2, y2 = r.get_llur()
img.pixtoradec(x1+o1.objx, y1+o1.objy)

In [ ]:
# Draw circles around all objects
Circle = canvas.get_draw_class('circle')
for obj in objs:
    x, y = x1+obj.objx, y1+obj.objy
    if r.contains(x, y):
        canvas.add(Circle(x, y, radius=10, color='yellow'))
        
# set pan and zoom to center
a.window.ginga_view.set_pan((x1+x2)/2, (y1+y2)/2)
a.window.ginga_view.scale_to(0.75, 0.75)

In [ ]:
a.window.ginga_view.show()

In [ ]:
a.close() #for ginga, there isn't an automatic window close for the HTML5 canvas